In [1]:
import matplotlib.pyplot as plt
import numpy as np


import SimpleITK as sitk
from scipy import interpolate

from back_transform_utils import crop_or_pad_Bern_new, skeleton_points
from utils import normalize_image_new

from skimage.morphology import skeletonize_3d, dilation, cube

from utils_centerline import interpolate_and_slice
from mayavi import mlab as mlab
from utils_centerline import extract_slice_from_sitk_image
%matplotlib inline

: 

In [ ]:
def crop_or_pad_normal_slices(data, new_shape):
    
    processed_data = np.zeros(new_shape)
    # axis 0 is the x-axis and we crop from top since aorta is at the bottom
    # axis 1 is the y-axis and we crop equally from both sides
    # axis 2 is the z-axis and we crop from the right (end of the image) since aorta is at the left
    delta_axis0 = data.shape[0] - new_shape[0]
    
    if len(new_shape) == 5: # Image
        # The x is always cropped, y always padded, z_cropped
        try:
            processed_data[:, :data.shape[1],:,:data.shape[3],... ] = data[delta_axis0:,:new_shape[1], :new_shape[2],...]
        except:
            processed_data[:, :data.shape[1],:,:data.shape[3],... ] = data[delta_axis0:,:new_shape[1], :new_shape[2],:new_shape[3],...]

    if len(new_shape) == 4: # Label
        # The x is always cropped, y always padded, z_cropped
        try:
            processed_data[:, :data.shape[1],:,:data.shape[3],... ] = data[delta_axis0:,:new_shape[1], :new_shape[2],...]
        except:
            processed_data[:, :data.shape[1],:,:data.shape[3],... ] = data[delta_axis0:,:new_shape[1], :new_shape[2],:new_shape[3],...]
    return processed_data

In [ ]:
# The output is bascially the anomaly score 
output = np.load("/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/4dflowmri_anomaly_detection/Results/Evaluation/cond_vae/masked_slice/20230712-1205_cond_vae_masked_slice_SSL_lr1.000e-03-e1200-bs8-gf_dim8-daFalse-n_experts3_2Dslice_decreased_interpolation_factor_cube_3/test/outputs/MACDAVD_311__anomaly_scores.npy")
seg_path = "/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/final_segmentations/controls/seg_MACDAVD_101_.npy"
img_path = f'/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/preprocessed/controls/numpy/MACDAVD_101_.npy'
output_reshaped = np.repeat(output.transpose(2,3,0,4,1), 4, axis=4)

# Load the geometry information of the slices 
geometry_dict = np.load("/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/4dflowmri_anomaly_detection/data/geometry_for_backtransformation/MACDAVD_101_.npy", allow_pickle=True).item()

output.shape, output_reshaped.shape



NameError: name 'np' is not defined

In [42]:
image = np.load(img_path)
segmented_original = np.load(seg_path)
# Enlarge the segmentation slightly to be sure that there are no cutoffs of the aorta
time_steps = segmented_original.shape[3]
segmented = dilation(segmented_original[:,:,:,3], cube(3))
temp_for_stack = [segmented for i in range(time_steps)]
segmented = np.stack(temp_for_stack, axis=3)

In [43]:
image.shape, segmented.shape

((160, 110, 60, 23, 4), (160, 110, 60, 23))

In [44]:
# We need to pad it back to original 36,36,64 - it was down to 32 for network reasons 

def expand_normal_slices(data, original_shape):
    
    # Create an array of zeros with the original shape
    expanded_data = np.zeros(original_shape)

    # Compute the difference in the first two dimensions
    delta_axis0 = original_shape[0] - data.shape[0]
    delta_axis1 = original_shape[1] - data.shape[1]
    
    # Place the cropped data back into the array
    expanded_data[delta_axis0:,:data.shape[1], :data.shape[2],:data.shape[3],... ] = data

    return expanded_data

output_reshaped_again = expand_normal_slices(output_reshaped, [36,36,64,24,4])


In [45]:
# Shape of the sliced images
common_image_shape = [36, 36, 64, 24, 4] # [x, y, z, t, num_channels]
end_shape = [32, 32, 64, 24, 4]
image = np.load(img_path)
segmented_original = np.load(seg_path)
# Enlarge the segmentation slightly to be sure that there are no cutoffs of the aorta
time_steps = segmented_original.shape[3]
segmented = dilation(segmented_original[:,:,:,3], cube(3))
temp_for_stack = [segmented for i in range(time_steps)]
segmented = np.stack(temp_for_stack, axis=3)

# normalize image to -1 to 1
image = normalize_image_new(image)
seg_shape = list(segmented.shape)
seg_shape.append(image.shape[-1])
#image_cropped = crop_or_pad_Bern_new(image, seg_shape)
image_cropped = image
temp_images_intensity = image_cropped[:,:,:,:,0] * segmented # change these back if it works
temp_images_vx = image_cropped[:,:,:,:,1] * segmented
temp_images_vy = image_cropped[:,:,:,:,2] * segmented
temp_images_vz = image_cropped[:,:,:,:,3] * segmented

## recombine the images
image = np.stack([temp_images_intensity,temp_images_vx,temp_images_vy,temp_images_vz], axis=4)
cnn_predictions = True



if cnn_predictions:
    points_ = skeleton_points(segmented_original, dilation_k = 0)
    points_dilated = skeleton_points(segmented_original, dilation_k = 4,erosion_k = 4)
else:
    points_ = skeleton_points(segmented_original, dilation_k = 0)
    points_dilated = skeleton_points(segmented_original, dilation_k = 2,erosion_k = 2)
points_all = points_dilated.copy()     


points = points_all[np.where(points_all[:,1]<60)]
points = points[np.where(points[:,0]<90)]


# Order the points in ascending order with x
points = points[points[:,0].argsort()[::-1]]

temp = []
for index, element in enumerate(points[5:]):
    if (index%5)==0:
        temp.append(element)


coords = np.array(temp)
# We create Slices across time and channels in a double for loop
temp_for_channel_stacking = []


geometry_dict = {}
for channel in range(image.shape[4]):
    print(f"Channel {channel}")

    temp_for_time_stacking = []
    for t in range(image.shape[3]):
        
        x = coords[:,0]
        y = coords[:,1]
        z = coords[:,2]

        coords_zyx = np.array([z,y,x]).transpose([1,0])
        

        #convert the image to SITK (here let's use the intensity for now)
        sitk_image = sitk.GetImageFromArray(image[:,:,:,t,channel])

        # spline parametrization
        #params = [0]# [i / (common_image_shape[2] - 1) for i in range(common_image_shape[2])]
        params =  [i / (common_image_shape[2] - 1) for i in range(common_image_shape[2])]
        tck, _ = interpolate.splprep(np.swapaxes(coords_zyx, 0, 1), k=3, s=200)

        # derivative is tangent to the curve
        points = np.swapaxes(interpolate.splev(params, tck, der=0), 0, 1)
        Zs = np.swapaxes(interpolate.splev(params, tck, der=1), 0, 1)
        direc = np.array(sitk_image.GetDirection()[3:6])
        slices = []
        orig_frames = []
        new_frames = []
        phys_sizes = []
        new_origins = []
        rotation_centers = []
        new_sizes = []
        rot_matrices = []
        orig_spacings = []
        orig_directions = []
        transforms = []
        new_origins_rotated = []
        Xs = []
        itk_resampled_images = []

        for i in range(len(Zs)):
            
            
            
            
            # I define the x'-vector as the projection of the y-vector onto the plane perpendicular to the spline
            xs = (direc - np.dot(direc, Zs[i]) / (np.power(np.linalg.norm(Zs[i]), 2)) * Zs[i])
            

            point, Z, X, new_size = points[i], Zs[i], xs, common_image_shape[:2] + [1]
            Xs.append(xs)
            num_dim = sitk_image.GetDimension()

            orig_pixelid = sitk_image.GetPixelIDValue()
            orig_direction = sitk_image.GetDirection()
            orig_spacing = np.array(sitk_image.GetSpacing())
            
            new_size = [int(el) for el in new_size]  # SimpleITK expects lists, not ndarrays
            point = [float(el) for el in point] # [z,y,x]
            

            rotation_center = sitk_image.TransformContinuousIndexToPhysicalPoint(point)

            X = X / np.linalg.norm(X)
            Z = Z / np.linalg.norm(Z)
            assert np.dot(X, Z) < 1e-12, 'the two input vectors are not perpendicular!'
            Y = np.cross(Z, X)
            #[print('orig_direction: ', orig_direction) if i == len(Zs)-1 else None]
            orig_frame = np.array(orig_direction).reshape(num_dim, num_dim)
            #[print('orig_frame: ', orig_frame) if i == len(Zs)-1 else None]
            new_frame = np.array([X, Y, Z])
            #[print('new_frame: ', new_frame) if i == len(Zs)-1 else None]

            # important: when resampling images, the transform is used to map points from the output image space into the input image space
            rot_matrix = np.dot(orig_frame, np.linalg.pinv(new_frame))
            transform = sitk.AffineTransform(rot_matrix.flatten(), np.zeros(num_dim), rotation_center)    

            phys_size = new_size * orig_spacing
            new_origin = rotation_center - phys_size / 2
            #[print('new_origin: ', new_origin) if i == len(Zs)-1 else None]

            resample_filter = sitk.ResampleImageFilter()
            resample_filter.SetSize(new_size)
            resample_filter.SetOutputSpacing(orig_spacing)
            resample_filter.SetOutputDirection(orig_direction)
            resample_filter.SetOutputOrigin(new_origin)
            resample_filter.SetInterpolator(sitk.sitkLinear)
            resample_filter.SetTransform(transform)
            resample_filter.SetDefaultPixelValue(0)

            resampled_sitk_image = resample_filter.Execute(sitk_image)
            if channel == 0 and t == 0:
                geometry_dict[f"slice_{i}_origin"] = resampled_sitk_image.GetOrigin()
                geometry_dict[f"slice_{i}_transform"] = transform

            
            
            np_resampled_sitk_image = sitk.GetArrayFromImage(resampled_sitk_image)
            itk_resampled_images.append(resampled_sitk_image)
            #[print(np_resampled_sitk_image.shape) if i == len(Zs)-1 else None]
            np_resampled_sitk_image = np_resampled_sitk_image.transpose([2,1,0])
            #[print(np_resampled_sitk_image.shape) if i == len(Zs)-1 else None]
            slices.append(np_resampled_sitk_image)
            orig_frames.append(orig_frame)
            new_frames.append(new_frame)
            phys_sizes.append(phys_size)
            new_origins.append(new_origin)
            rotation_centers.append(rotation_center)
            new_sizes.append(new_size)
            rot_matrices.append(rot_matrix)
            orig_spacings.append(orig_spacing)
            orig_directions.append(orig_direction)
            transforms.append(transform)
            
        # stick slices together
        straightened = np.concatenate(slices, axis=2)
        
        #straightened = interpolate_and_slice(image[:,:,:,t,channel], coords, common_image_shape)
        temp_for_time_stacking.append(straightened)

    channel_stacked = np.stack(temp_for_time_stacking, axis=-1)
    temp_for_channel_stacking.append(channel_stacked)

straightened = np.stack(temp_for_channel_stacking, axis=-1)

/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/back_transform_utils.py:27: FutureWarning: `selem` is a deprecated argument name for `binary_erosion`. It will be removed in version 1.0. Please use `footprint` instead.
  avg = binary_erosion(avg, selem=np.ones((erosion_k, erosion_k,erosion_k)))
/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/back_transform_utils.py:28: FutureWarning: `selem` is a deprecated argument name for `dilation`. It will be removed in version 1.0. Please use `footprint` instead.
  avg = dilation(avg, selem=np.ones((dilation_k, dilation_k,dilation_k)))


Channel 0
Channel 1
Channel 2
Channel 3


In [46]:
print("Image shape before cropping and padding:" + str(straightened.shape))
straightened = crop_or_pad_normal_slices(straightened, end_shape)
print("Image shape after cropping and padding:" + str(straightened.shape))

Image shape before cropping and padding:(36, 36, 64, 23, 4)
Image shape after cropping and padding:(32, 32, 64, 24, 4)


In [47]:
image = np.load(img_path)
image.shape

(160, 110, 60, 23, 4)

In [48]:
# The geometric information we need from the initial image does not change with channel or time 
sitk_image = sitk.GetImageFromArray(image[:,:,:,0,0])


In [49]:
# Let us do the back transform for each time step and slice now
temp_for_time_stacking = []
for t in range(output_reshaped_again.shape[3]):
    temp_for_slice_stacking = []
    for slice_i in range(output_reshaped_again.shape[2]):
        resampled_sitk_image = sitk.GetImageFromArray(output_reshaped_again[:,:,slice_i:slice_i+1,t,0].transpose([2,1,0]))
        # You need to set the origin, the direction and the spacing of the resampled image
        resampled_sitk_image.SetOrigin(geometry_dict[f"slice_{slice_i}_origin"])
        resampled_sitk_image.SetDirection((1,0,0,0,1,0,0,0,1))
        resampled_sitk_image.SetSpacing((1,1,1))
        inverse_transform = geometry_dict[f"slice_{slice_i}_transform"].GetInverse()
        resampler = sitk.ResampleImageFilter()
        resampler.SetSize(sitk_image.GetSize()) # set to original image size
        resampler.SetOutputSpacing(sitk_image.GetSpacing()) # set to original image spacing
        resampler.SetOutputDirection(sitk_image.GetDirection()) # set to original image direction
        resampler.SetOutputOrigin(sitk_image.GetOrigin()) # set to original image origin
        resampler.SetInterpolator(sitk.sitkLinear) 
        resampler.SetTransform(inverse_transform) 

        resampled_slice = resampler.Execute(resampled_sitk_image) # apply to the resampled slice
        resampled_slice = sitk.GetArrayFromImage(resampled_slice)
        temp_for_slice_stacking.append(resampled_slice)
        
    temp_for_time_stacking.append(np.sum(np.array(temp_for_slice_stacking), axis=0))
        
anomaly_score_4d_image = np.stack(temp_for_time_stacking, axis=-1)

In [50]:
len(temp_for_slice_stacking)

64

In [51]:
#  White background
mlab.figure(bgcolor=(1, 1, 1), size=(800, 800))


anomaly_score_color = mlab.contour3d(-anomaly_score_4d_image[...,5], colormap = 'cool', opacity = 0.1)
anomaly_score_color = mlab.contour3d(-temp_for_slice_stacking[0], colormap = 'cool', opacity = 0.6)
mlab.colorbar(anomaly_score_color, orientation='vertical')
mlab.contour3d(-temp_for_slice_stacking[5], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[5], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[10], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[15], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[20], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[25], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[30], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[35], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[40], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[45], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[50], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[55], colormap = 'cool', opacity = 0.6)
mlab.contour3d(-temp_for_slice_stacking[60], colormap = 'cool', opacity = 0.6)


mlab.contour3d(segmented[...,0], colormap = 'gray', opacity = 0.3)
# Background color should be black

#mlab.axes(xlabel='X', ylabel='Y', zlabel='Z') #Display axis
# Remove axis
mlab.show()


In [22]:
anomaly_score_4d_image.shape
from tvtk.api import tvtk, write_data
import os

In [23]:
patient_type = "patients" # "controls" or "patients"
images_path = f"/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/preprocessed/{patient_type}/numpy_velocity_scaled"
segs_path = f"/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/final_segmentations/{patient_type}"
subject_id = "208"
image_path = os.path.join(images_path, f"MACDAVD_{subject_id}_.npy")
seg_path = os.path.join(segs_path, f"seg_MACDAVD_{subject_id}_.npy")

In [24]:
np.load(image_path).shape ,anomaly_score_4d_image.shape

((160, 110, 56, 15, 4), (160, 110, 56, 24))

In [32]:
anomaly_score_4d_image[...,2].max()

0.9134607855756806

In [28]:
time_slices = np.shape(anomaly_score_4d_image)[3]
file_name = "MACDAVD_208_"
for time_slice in np.arange(time_slices):


    # Magnitude

    anomaly_score = anomaly_score_4d_image[:,:,:,time_slice]

    # X,Y,Z
    dim=anomaly_score.shape
    # Generate the grid
    xx,yy,zz=np.mgrid[0:dim[0],0:dim[1],0:dim[2]]
    
    pts_anomaly_score = np.empty(anomaly_score.shape + (3,), dtype=int)
    pts_anomaly_score[..., 0] = xx
    pts_anomaly_score[..., 1] = yy
    pts_anomaly_score[..., 2] = zz

    pts_anomaly_score = pts_anomaly_score.transpose(2, 1, 0, 3).copy()
    pts_anomaly_score.shape = pts_anomaly_score.size // 3, 3
    anomaly_score_vec = np.empty(anomaly_score.shape + (1,), dtype=float)
    anomaly_score_vec[..., 0] = anomaly_score

    anomaly_score_vec = anomaly_score_vec.transpose(2, 1, 0, 3).copy()
    
    anomaly_score_vec.shape = anomaly_score_vec.size
    
    
    anomaly_score_grid = tvtk.StructuredGrid(dimensions=xx.shape, points=pts_anomaly_score)
    anomaly_score_grid.point_data.scalars = anomaly_score_vec
    anomaly_score_grid.point_data.scalars.name = 'anomaly_score_intensity'
    #anomaly_score_grid.cell_data.scalars = anomaly_score_vec[:-1]
    #anomaly_score_grid.cell_data.scalars.name = 'anomaly_score_intensity'
    write_data(anomaly_score_grid, os.path.join("anomaly_score/",f"point_again_{file_name}_anomaly_score_t{time_slice}.vtk"))

dimdim

## Cool it seems to work for the whole thing now let us implement it in the pipeline

In [31]:
dim

(160, 110, 56)

### Below we have an example, now we want for the whole pipeline

In [19]:
# Shape of the sliced images
common_image_shape = [36, 36, 64, 24, 4] # [x, y, z, t, num_channels]

image = np.load(img_path)
segmented_original = np.load(seg_path)
# Enlarge the segmentation slightly to be sure that there are no cutoffs of the aorta
time_steps = segmented_original.shape[3]
segmented = dilation(segmented_original[:,:,:,3], cube(3))
temp_for_stack = [segmented for i in range(time_steps)]
segmented = np.stack(temp_for_stack, axis=3)

# normalize image to -1 to 1
image = normalize_image_new(image)
seg_shape = list(segmented.shape)
seg_shape.append(image.shape[-1])
image_cropped = crop_or_pad_Bern_new(image, seg_shape)
temp_images_intensity = image_cropped[:,:,:,:,0] * segmented # change these back if it works
temp_images_vx = image_cropped[:,:,:,:,1] * segmented
temp_images_vy = image_cropped[:,:,:,:,2] * segmented
temp_images_vz = image_cropped[:,:,:,:,3] * segmented

## recombine the images
image = np.stack([temp_images_intensity,temp_images_vx,temp_images_vy,temp_images_vz], axis=4)
cnn_predictions = False



if cnn_predictions:
    points_ = skeleton_points(segmented_original, dilation_k = 0)
    points_dilated = skeleton_points(segmented_original, dilation_k = 4,erosion_k = 4)
else:
    points_ = skeleton_points(segmented_original, dilation_k = 0)
    points_dilated = skeleton_points(segmented_original, dilation_k = 2,erosion_k = 2)
points_all = points_dilated.copy()     


points = points_all[np.where(points_all[:,1]<60)]
points = points[np.where(points[:,0]<90)]


# Order the points in ascending order with x
points = points[points[:,0].argsort()[::-1]]

temp = []
for index, element in enumerate(points[5:]):
    if (index%5)==0:
        temp.append(element)


coords = np.array(temp)




common_image_shape = [36, 36, 64, 24, 4] # [x, y, z, t, num_channels]
image_spatial_ex = image[:,:,:,0,0]
x = coords[:,0]
y = coords[:,1]
z = coords[:,2]

coords_zyx = np.array([z,y,x]).transpose([1,0])
#coords_zyx = np.array([x,y,z]).transpose([1,0])
print(coords_zyx.shape)

#convert the image to SITK (here let's use the intensity for now)
sitk_image = sitk.GetImageFromArray(image_spatial_ex[:,:,:])

# spline parametrization
#params = [0]# [i / (common_image_shape[2] - 1) for i in range(common_image_shape[2])]
params =  [i / (common_image_shape[2] - 1) for i in range(common_image_shape[2])]
tck, _ = interpolate.splprep(np.swapaxes(coords_zyx, 0, 1), k=3, s=200)

# derivative is tangent to the curve
points = np.swapaxes(interpolate.splev(params, tck, der=0), 0, 1)
Zs = np.swapaxes(interpolate.splev(params, tck, der=1), 0, 1)
direc = np.array(sitk_image.GetDirection()[3:6])
#plt.scatter(points[:,2], points[:,1])
#plt.scatter(x, y)
#plt.show()
slices = []
orig_frames = []
new_frames = []
phys_sizes = []
new_origins = []
rotation_centers = []
new_sizes = []
rot_matrices = []
orig_spacings = []
orig_directions = []
transforms = []
new_origins_rotated = []
Xs = []
itk_resampled_images = []

for i in range(len(Zs)):
    
    # I define the x'-vector as the projection of the y-vector onto the plane perpendicular to the spline
    xs = (direc - np.dot(direc, Zs[i]) / (np.power(np.linalg.norm(Zs[i]), 2)) * Zs[i])
    

    point, Z, X, new_size = points[i], Zs[i], xs, common_image_shape[:2] + [1]
    Xs.append(xs)
    num_dim = sitk_image.GetDimension()

    orig_pixelid = sitk_image.GetPixelIDValue()
    orig_direction = sitk_image.GetDirection()
    orig_spacing = np.array(sitk_image.GetSpacing())
    
    new_size = [int(el) for el in new_size]  # SimpleITK expects lists, not ndarrays
    point = [float(el) for el in point] # [z,y,x]
    

    rotation_center = sitk_image.TransformContinuousIndexToPhysicalPoint(point)

    X = X / np.linalg.norm(X)
    Z = Z / np.linalg.norm(Z)
    assert np.dot(X, Z) < 1e-12, 'the two input vectors are not perpendicular!'
    Y = np.cross(Z, X)
    [print('orig_direction: ', orig_direction) if i == len(Zs)-1 else None]
    orig_frame = np.array(orig_direction).reshape(num_dim, num_dim)
    [print('orig_frame: ', orig_frame) if i == len(Zs)-1 else None]
    new_frame = np.array([X, Y, Z])
    [print('new_frame: ', new_frame) if i == len(Zs)-1 else None]

    # important: when resampling images, the transform is used to map points from the output image space into the input image space
    rot_matrix = np.dot(orig_frame, np.linalg.pinv(new_frame))
    transform = sitk.AffineTransform(rot_matrix.flatten(), np.zeros(num_dim), rotation_center)
    #transform = sitk.AffineTransform(np.eye(3).flatten(), np.zeros(num_dim), rotation_center)
    #transform = sitk.AffineTransform(rot_matrix.flatten(), np.zeros(num_dim), [0,0,0])

    

    phys_size = new_size * orig_spacing
    new_origin = rotation_center - phys_size / 2
    [print('new_origin: ', new_origin) if i == len(Zs)-1 else None]

    # Apply rotation on new_origin
    #new_origin_rotated = np.dot(rot_matrix, new_origin - rotation_center) + rotation_center

    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetSize(new_size)
    resample_filter.SetOutputSpacing(orig_spacing)
    resample_filter.SetOutputDirection(orig_direction)
    resample_filter.SetOutputOrigin(new_origin)
    resample_filter.SetInterpolator(sitk.sitkLinear)
    resample_filter.SetTransform(transform)
    resample_filter.SetDefaultPixelValue(0)

    resampled_sitk_image = resample_filter.Execute(sitk_image)
    
    #sitk_slice, new_size, orig_spacing, orig_direction, orig_frame, new_frame, phys_size, new_origin, rotation_center,rot_matrix, transform = extract_slice_from_sitk_image_extended(sitk_image, points[i], Zs[i], xs, list(common_image_shape[:2]) + [1], fill_value=0)
    np_resampled_sitk_image = sitk.GetArrayFromImage(resampled_sitk_image)
    itk_resampled_images.append(resampled_sitk_image)
    [print(np_resampled_sitk_image.shape) if i == len(Zs)-1 else None]
    np_resampled_sitk_image = np_resampled_sitk_image.transpose([2,1,0])
    [print(np_resampled_sitk_image.shape) if i == len(Zs)-1 else None]
    slices.append(np_resampled_sitk_image)
    orig_frames.append(orig_frame)
    new_frames.append(new_frame)
    phys_sizes.append(phys_size)
    new_origins.append(new_origin)
    rotation_centers.append(rotation_center)
    new_sizes.append(new_size)
    rot_matrices.append(rot_matrix)
    orig_spacings.append(orig_spacing)
    orig_directions.append(orig_direction)
    transforms.append(transform)
    
# stick slices together
image_slices = np.concatenate(slices, axis=2)
print(image_slices.shape)


Check this function!


/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/back_transform_utils.py:27: FutureWarning: `selem` is a deprecated argument name for `binary_erosion`. It will be removed in version 1.0. Please use `footprint` instead.
  avg = binary_erosion(avg, selem=np.ones((erosion_k, erosion_k,erosion_k)))
/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/back_transform_utils.py:28: FutureWarning: `selem` is a deprecated argument name for `dilation`. It will be removed in version 1.0. Please use `footprint` instead.
  avg = dilation(avg, selem=np.ones((dilation_k, dilation_k,dilation_k)))


(10, 3)
orig_direction:  (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
orig_frame:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
new_frame:  [[-0.4140637   0.8540037   0.31500624]
 [ 0.60547042  0.          0.79586781]
 [ 0.67967405  0.52026693 -0.51707398]]
new_origin:  [ 6.18573314 37.5803668  24.18380119]
(1, 36, 36)
(36, 36, 1)
(36, 36, 64)


In [20]:
resampled_slices = []
for slice_i in np.arange(64):
    
    resampled_sitk_image = sitk.GetImageFromArray(output_reshaped_again[:,:,slice_i:slice_i+1,0,0].transpose([2,1,0]))
    # You need to set the origin, the direction and the spacing of the resampled image
    resampled_sitk_image.SetOrigin(itk_resampled_images[slice_i].GetOrigin())
    resampled_sitk_image.SetDirection(itk_resampled_images[slice_i].GetDirection())
    resampled_sitk_image.SetSpacing(itk_resampled_images[slice_i].GetSpacing())
    inverse_transform = transforms[slice_i].GetInverse()
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(sitk_image.GetSize()) # set to original image size
    resampler.SetOutputSpacing(sitk_image.GetSpacing()) # set to original image spacing
    resampler.SetOutputDirection(sitk_image.GetDirection()) # set to original image direction
    resampler.SetOutputOrigin(sitk_image.GetOrigin()) # set to original image origin
    resampler.SetInterpolator(sitk.sitkLinear) 
    resampler.SetTransform(inverse_transform) 

    resampled_slice = resampler.Execute(resampled_sitk_image) # apply to the resampled slice
    resampled_slice = sitk.GetArrayFromImage(resampled_slice)
    resampled_slices.append(resampled_slice)

In [21]:
# Bascially now you have 64 times the orginial image where for each it corresponds to the anomaly score of the slice
# So we just sum across the 64 images to get the anomaly score for the whole image
anomaly_score_image = np.sum(np.array(resampled_slices), axis=0)
anomaly_score_image.shape

(144, 112, 40)

In [22]:
mlab.figure()


anomaly_score_color = mlab.contour3d(anomaly_score_image, colormap = 'cool', opacity = 0.1)
mlab.colorbar(anomaly_score_color, orientation='vertical')
mlab.contour3d(segmented[...,0], colormap = 'gray', opacity = 0.1)
mlab.axes(xlabel='X', ylabel='Y', zlabel='Z') #Display axis
mlab.show()
